Importing all the neccessary libraries to build the sentiment analysis AI agent

The model will be built using langchain and langgraph because it's the most used library to build AI agents worldwideeeee

In [1]:
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence,List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from operator import add as add_messages

Importing two llama3 llms with a temperature =0 to avoid hallacunation using Ollama
Keep in mind the llm1 and llm2 can we swapped with any llm

In [2]:
llm1 = OllamaLLM(model="llama3",temperature=0)
sentiment_prompt = """You are an AI that performs sentiment analysis on a given text.
Please output only the sentiment: either "positive" or "negative", without any other text or explanation."""

llm2 = OllamaLLM(model="llama3",temperature=0)
accuracy_prompt = """You are an AI that assesses the accuracy of a sentiment analysis.
You will be given a list of original texts and their sentiment analysis results.
Analyze if each sentiment is correct. Finally, calculate the overall accuracy as a percentage
and output only the percentage number (e.g., 75%), also keep in mind that the sentiment generated by the sentiment analysis
is postive or negative it doesn't have neutral
"""

The state is used to store context of llm inputs and outputs in langgraph

In [3]:
class AgentState(TypedDict):
    input_texts: List[str]
    current_index: int
    sentiment_results: List[str]
    accuracy_score: str

This function takes the sentiment from the state and invoke it to the llm1 and the llm1 generates whether the sentiment is positive or negative

In [4]:
def sentiment_analysis_node(state: AgentState) -> AgentState:
    """This node performs sentiment analysis using LLM1."""
    
    current_index = state['current_index']
    texts_to_analyze = state['input_texts']
    
    # Check if there are texts left to analyze
    if current_index >= len(texts_to_analyze):
        print("All texts have been analyzed. Transitioning to accuracy analysis.")
        return state # The graph's router will handle the transition
    
    current_text = texts_to_analyze[current_index]
    
    print(f"Analyzing text {current_index + 1}: '{current_text}'")
    
    messages = [
        SystemMessage(content=sentiment_prompt),
        HumanMessage(content=current_text)
    ]
    
    # Invoke LLM1 and get the sentiment
    sentiment_result = llm1.invoke(messages).strip()
    
    # Append the result to the state
    state['sentiment_results'].append(sentiment_result)
    state['current_index'] += 1
    
    print(f"  -> Result: {sentiment_result}")
    
    return state

The llm2 in this function critiques the sentiments generated from llm1 and generates the accuracy and the correct answer

In [5]:
def accuracy_analysis_node(state: AgentState) -> AgentState:
    """This node calculates the accuracy score using LLM2."""
    
    print("\n--- Starting Accuracy Analysis ---")
    
    # Prepare the prompt for LLM2 with all data
    analysis_data = "Original Texts and Sentiments:\n"
    for i, text in enumerate(state['input_texts']):
        analysis_data += f"{i+1}. Text: '{text}' | Sentiment: {state['sentiment_results'][i]}\n"
    
    messages = [
        SystemMessage(content=accuracy_prompt),
        HumanMessage(content=analysis_data)
    ]
    
    # Invoke LLM2 and get the final accuracy score
    accuracy_score = llm2.invoke(messages).strip()
    
    state['accuracy_score'] = accuracy_score
    
    print(f"Final Accuracy Score: {accuracy_score}")
    
    return state

This function takes the input list of sentiments and decided whether to stop the process or continue invoking the llm1 and llm2.
For example if there are 4 inputs in the list the agent will not stop working until all 4 sentiments are generated and their accuracy.
This function works as a router which routes to different actions based on conditions

In [6]:
def router(state: AgentState) -> str:
    """A router to decide whether to loop or move to accuracy analysis."""
    
    # Get the number of texts and the current index
    num_texts = len(state['input_texts'])
    current_index = state['current_index']
    
    # If the counter is less than the total number of texts, loop back
    if current_index < num_texts:
        return "continue_loop"
    
    # Otherwise, move to the accuracy analysis node
    return "analyze_accuracy"

# Build the LangGraph workflow
workflow = StateGraph(AgentState)

# Add the nodes to the graph
workflow.add_node("sentiment_analysis", sentiment_analysis_node)
workflow.add_node("accuracy_analysis", accuracy_analysis_node)

# Set the entry point and define the conditional edge
workflow.set_entry_point("sentiment_analysis")
workflow.add_conditional_edges(
    "sentiment_analysis",
    router,
    {
        "continue_loop": "sentiment_analysis",  # Loop back to the same node
        "analyze_accuracy": "accuracy_analysis" # Move to the next node
    }
)

# Add the final edge to end the graph
workflow.add_edge("accuracy_analysis", END)

# Compile the graph
app = workflow.compile()

Experimenting with the agent to see whether it's working

In [7]:
# Example data for the agent to process
initial_data = [
    "I love this new phone, it's so fast!",
    "The customer service was terrible, I'm very disappointed.",
    "This movie was okay, but not the best.",
    "The new restaurant has amazing food and great prices."
]

# Run the graph with the initial state
final_state = app.invoke({
    "input_texts": initial_data,
    "current_index": 0,
    "sentiment_results": [],
    "accuracy_score": ""
})

# Print the final results
print("\n--- Final Output ---")
print(f"Original Texts: {final_state['input_texts']}")
print(f"Sentiment Results: {final_state['sentiment_results']}")
print(f"Calculated Accuracy Score: {final_state['accuracy_score']}")

Analyzing text 1: 'I love this new phone, it's so fast!'


ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it